# CACM DATASET

## distribute data

### data info

In [ ]:
CACMDATA = '../../../cacm/cacm.all'
import re
IDMarker = re.compile('(\.I.)')
allMarkers = re.compile('(\.[ITWBACKNX] )')

### queries info

In [ ]:
CACMQUERY = '../../../cacm/query.text'
CACMQRELS = '../../../cacm/qrels.text'
import re
queryMarkers = re.compile('(\.[IWAN] )')

In [ ]:
def getData(PATH, marker):
    """get the data from the file and split it by ID"""
    with open(PATH, 'r') as f:
        t = f.read().replace('\n', ' ')
        lines = re.split(marker, t)
        lines.pop(0)
    return lines

### converte CACM.all

In [ ]:
cacmData = getData(CACMDATA, allMarkers)

In [ ]:
import pandas as pd
dataFrame = pd.DataFrame(columns=['.I','.T','.W','.B','.A','.K','.C','.N','.X'])
seriesDict:dict = {
    '.I': None,
    '.T': None,
    '.W': None,
    '.B': None,
    '.K': None,
    '.C': None,
    '.A': None,
    '.N': None,
    '.X': None
}
seriesData = seriesDict.copy()
notTheFirst = False
for i in range(0, len(cacmData), 2):
    if (notTheFirst and cacmData[i].strip() == '.I'):
        dataFrame = dataFrame.append(seriesData, ignore_index=True)
        seriesData = seriesDict.copy()
    
    seriesData[cacmData[i].strip()] = cacmData[i+1].strip()
    notTheFirst = True
dataFrame = dataFrame.append(seriesData, ignore_index=True)
dataFrame.set_index('.I', inplace=True)
dataFrame.head()

In [ ]:
dataFrame.to_csv('../../cacmData/cacmCsv.csv')

### converte query.text

In [ ]:
cacmQuery = getData(CACMQUERY, queryMarkers)

In [ ]:
import pandas as pd
qDataFrame = pd.DataFrame(columns=['.I','.W','.A','.N'])
seriesDict:dict = {
    '.I': None,
    '.W': None,
    '.A': None,
    '.N': None
}
seriesData = seriesDict.copy()
notTheFirst = False
for i in range(0, len(cacmQuery), 2):
    if (notTheFirst and cacmQuery[i].strip() == '.I'):
        qDataFrame = qDataFrame.append(seriesData, ignore_index=True)
        seriesData = seriesDict.copy()
    
    seriesData[cacmQuery[i].strip()] = cacmQuery[i+1].strip()
    notTheFirst = True
qDataFrame = qDataFrame.append(seriesData, ignore_index=True)
qDataFrame.set_index('.I', inplace=True)
qDataFrame.head()

In [ ]:
qDataFrame.to_csv('../../cacmData/cacmQueryCsv.csv')

### converte qrels.text

In [ ]:
import pandas as pd

def getRles(path):
    with open(path, 'r') as f:
        global qrlesList
        qrlesList = f.read().split('\n')
        return qrlesList

qrelsData = getRles(CACMQRELS)
qrelsFrame = pd.DataFrame(columns=['.I', 'data'])
seriesDict:dict = {'.I':None, 'data':None}
seriesData = seriesDict.copy()
for i in qrelsData:
    try:
        element = i.split(' ')
        seriesData['.I'] = int(element[0])
        seriesData['data'] = int(element[1])
        qrelsFrame = qrelsFrame.append(seriesData, ignore_index=True)
    except:
        pass
qrelsFrame.head()

In [ ]:
qrelsFrame.to_csv('../../cacmData/cacmQRels.csv')

## clean preproccesing

### CACM

In [1]:
import pandas as pd
df:pd.DataFrame = pd.read_csv('../../cacmData/cacmCsv.csv', index_col='.I')
df.head()

,.T,.W,.B,.A,.K,.C,.N,.X
.I,,,,,,,,
1,Preliminary Report-International Algebraic Lan...,NaN,"CACM December, 1958","Perlis, A. J. Samelson,K.",NaN,NaN,"CA581203 JB March 22, 1978 8:28 PM",100\t5\t1 123\t5\t1 164\t5\t1 1\t5\t1 1\t5\t1 ...
2,Extraction of Roots by Repeated Subtractions f...,NaN,"CACM December, 1958","Sugai, I.",NaN,NaN,"CA581202 JB March 22, 1978 8:29 PM",2\t5\t2 2\t5\t2 2\t5\t2
3,Techniques Department on Matrix Program Schemes,NaN,"CACM December, 1958","Friedman, M. D.",NaN,NaN,"CA581201 JB March 22, 1978 8:30 PM",3\t5\t3 3\t5\t3 3\t5\t3
4,Glossary of Computer Engineering and Programmi...,NaN,"CACM November, 1958",NaN,NaN,NaN,"CA581103 JB March 22, 1978 8:32 PM",4\t5\t4 4\t5\t4 4\t5\t4
5,Two Square-Root Approximations,NaN,"CACM November, 1958","Wadey, W. G.",NaN,NaN,"CA581102 JB March 22, 1978 8:33 PM",5\t5\t5 5\t5\t5 5\t5\t5


In [ ]:
df.describe()

In [ ]:
df.info()

In [2]:
print(df.loc[:,'.T'].isnull().value_counts(), end='\n\n')
print(df.loc[:,'.W'].isnull().value_counts(), end='\n\n')
print(df.loc[:,'.A'].isnull().value_counts())

False    3203
True        1
Name: .T, dtype: int64

True     1617
False    1587
Name: .W, dtype: int64

False    3120
True       84
Name: .A, dtype: int64


In [3]:
df.fillna('', inplace=True)

In [ ]:
df.duplicated().value_counts()

#### clean wrong values in .N

In [4]:
lis = df.loc[:,'.N'].to_list()
import re
lis = [s.replace(',',' ') for s in lis]
ids = []


pattern = r'[A-Z][a-z]+ +[0-9][0-9]? +[0-9]{4} +[0-9][0-9]?:[0-9][0-9]? +(PM|AM)'
for i in range(0,len(lis)):
    if re.search(pattern, lis[i]) == None:
        ids.append(i)


removePattern = r'CA[0-9]* ?[A-Z]{2} +[A-Z][a-z]+ +[0-9][0-9]? +[0-9]{4}'
for i in ids:
    lis[i] = re.findall(removePattern, lis[i])[0]

df.loc[:, '.N'] = lis

In [5]:
df.to_csv('../../cacmData/cacmDataCleaned.csv')

#### methods

###### lowercase

In [7]:
def toLower(text):
    return text.lower()

###### Numbers to words

In [8]:
import inflect
p = inflect.engine()

import re
reg = r'([0-9]+)'

def isFLoat(strNum):
    try:
        float(strNum)
        return True
    except:
        return False


def converteNumbers(text):
    tempText = text.split()
    newText = []
    for word in tempText:
        tempList = re.split(reg,word)
        for miniWord in tempList:
            if miniWord.isdigit() or isFLoat(miniWord):
                temp = p.number_to_words(miniWord)
                newText.append(removePunctuation(temp))
            else:
                newText.append(miniWord)        
    tempText = ' '.join(newText)
    return tempText
    # return text

###### remove punctuation

In [9]:
import string
translator = str.maketrans(string.punctuation,' '*len(string.punctuation))
def removePunctuation(text):
    global translator
    return text.translate(translator)


###### remove whitespaces

In [10]:
def removeWhiteSpace(text):
    return " ".join(text.split())

###### remove stop words

In [11]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def removeStopWords(text):
    sw = set(stopwords.words("english"))
    wt = word_tokenize(text)
    filteredText = [word for word in wt if word not in sw]
    return ' '.join(filteredText)
    # return text

###### stemming

In [19]:
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

stemmer = PorterStemmer()
def stemWords(text):
    global stemmer
    wt = word_tokenize(text)
    stems = []
    for word in wt:
        temp = stemmer.stem(word)
        # if not temp == word:
        #     temp = correctWords(temp)
        stems.append(temp)
    return ' '.join(stems)
    # return text

###### lemmatization

In [13]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk import pos_tag, defaultdict

lemmatizer = WordNetLemmatizer()

tag_map = defaultdict(lambda: wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV




def lemmatizeWords(text):
    # wt = word_tokenize(text)
    # lemmas = [lemmatizer.lemmatize(word, pos='a') for word in wt]
    # return ' '.join(lemmas)
    # return text

    tokens = word_tokenize(text)
    lmtzr = WordNetLemmatizer()
    lemmas = [lmtzr.lemmatize(token, tag_map[tag[0]]) for token, tag in pos_tag(tokens) ]
    return ' '.join(lemmas)
lemmatizeWords('hard	')

'hard'

###### correcting

In [26]:
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams
from nltk.corpus import words
correct_words = words.words()
incorrectWords = '''preliminari'''.split()
result = []
def correctWords(text):
    for word in text:
        try:
            temp = [(jaccard_distance(set(ngrams(word, 2)),
                                      set(ngrams(w, 2))),w)
                                      for w in correct_words if w[0] == word[0]]
            result.append(sorted(temp, key = lambda val:val[0])[0][1])
        except:
            pass
    return ' '.join(result)
# correctWords(incorrectWords)

#### cacm process

In [15]:
import pandas as pd
import re

caRegex = r'CA[0-9a-z]* ?[A-Z]+ '
def TitlePreProcesse(t):
    tempText = toLower(t)
    tempText = removePunctuation(tempText)
    tempText = converteNumbers(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = stemWords(tempText)
    tempText = lemmatizeWords(tempText)
    # tempText = correctWords(tempText)
    return tempText

def abstractPreProcesse(a):
    tempText = toLower(a)
    tempText = removePunctuation(tempText)
    tempText = converteNumbers(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = stemWords(tempText)
    tempText = lemmatizeWords(tempText)
    # tempText = correctWords(tempText)
    return tempText

def publicationPreProcesse(p):
    tempText = p.replace('CACM ','')
    return pd.to_datetime(tempText)

def authorPreProcesse(a):
    tempText = toLower(a)
    lis = tempText.split(' ')
    names = ' '
    l = []
    for word in lis:
      if ',' in word:
          l.append(removePunctuation(word.replace(',','')))
    names = ' '.join(l)
    return names

def kPreProcesse(k):
    tempText = toLower(k)
    tempText = removePunctuation(tempText)
    tempText = converteNumbers(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = stemWords(tempText)
    tempText = lemmatizeWords(tempText)
    # tempText = correctWords(tempText)
    return tempText

def cPreProcesse(c):
    tempText = removeWhiteSpace(c)
    return tempText

def infoPreProcesse(i):
    # tempText = toLower(i)
    # tempText = converteNumbers(tempText)
    # tempText = removePunctuation(tempText)
    # remove the (CA581202 JB) ... and converte the rest to date type
    # tempText = re.sub(caRegex, '', i)
    # tempText = pd.to_datetime(tempText)
    #tempText = removeWhiteSpace(tempText)
    # tempText = removeStopWords(tempText)
    # tempText = stemWords(tempText)
    # tempText = lemmatizeWords(tempText)
    return i

In [16]:
import pandas as pd
def preprocessedData(dataFrame:pd.DataFrame):
    pdataFrame = pd.DataFrame()
    seriesDict:dict = {} 
    seriesData = seriesDict.copy()
    for i in dataFrame.index:
        try:
            templist = []
            tempC = tempB = tempI = None
            if not dataFrame.loc[i, '.T'] == '':
                templist.append(TitlePreProcesse(dataFrame.loc[i, '.T']))
            if not dataFrame.loc[i, '.W'] == '':
                templist.append(abstractPreProcesse(dataFrame.loc[i, '.W']))
            if not dataFrame.loc[i, '.B'] == '':
                tempB = publicationPreProcesse(dataFrame.loc[i, '.B'])
            if not dataFrame.loc[i, '.A'] == '':
                templist.append(authorPreProcesse(dataFrame.loc[i, '.A']))
            # if not dataFrame.loc[i, '.K'] == '':
            #     templist.append(kPreProcesse(dataFrame.loc[i, '.K']))
            # if not dataFrame.loc[i, '.C'] == '':
            #     tempC = cPreProcesse(dataFrame.loc[i, '.C'])


            seriesData['.I'] = i
            seriesData['data'] = ' '.join(templist)
            seriesData['.B'] = tempB
            # seriesData['.C'] = tempC
            pdataFrame = pdataFrame.append(seriesData, ignore_index=True)
        except:
            print(i)
            raise 
    
    pdataFrame.set_index('.I', inplace=True)
    pdataFrame.fillna('', inplace=True)
    return pdataFrame


In [17]:
import pandas as pd
data = pd.read_csv('../../cacmData/cacmDataCleaned.csv')
data.set_index('.I', inplace=True)
data.fillna('', inplace=True)
data.head()

,.T,.W,.B,.A,.K,.C,.N,.X
.I,,,,,,,,
1,Preliminary Report-International Algebraic Lan...,,"CACM December, 1958","Perlis, A. J. Samelson,K.",,,CA581203 JB March 22 1978 8:28 PM,100\t5\t1 123\t5\t1 164\t5\t1 1\t5\t1 1\t5\t1 ...
2,Extraction of Roots by Repeated Subtractions f...,,"CACM December, 1958","Sugai, I.",,,CA581202 JB March 22 1978 8:29 PM,2\t5\t2 2\t5\t2 2\t5\t2
3,Techniques Department on Matrix Program Schemes,,"CACM December, 1958","Friedman, M. D.",,,CA581201 JB March 22 1978 8:30 PM,3\t5\t3 3\t5\t3 3\t5\t3
4,Glossary of Computer Engineering and Programmi...,,"CACM November, 1958",,,,CA581103 JB March 22 1978 8:32 PM,4\t5\t4 4\t5\t4 4\t5\t4
5,Two Square-Root Approximations,,"CACM November, 1958","Wadey, W. G.",,,CA581102 JB March 22 1978 8:33 PM,5\t5\t5 5\t5\t5 5\t5\t5


In [27]:
processedDAta = preprocessedData(data)
processedDAta.head()

,data,.B
.I,,
1.0,preliminari report intern algebra languag perl...,1958-12-01
2.0,extract root repeat subtract digit comput sugai,1958-12-01
3.0,techniqu depart matrix program scheme friedman,1958-12-01
4.0,glossari comput engin program terminolog,1958-11-01
5.0,two squar root approxim wadey,1958-11-01


In [21]:
processedDAta.to_csv('../../cacmData/cacmDataPreprocessed.csv')

#### query process

In [29]:
import pandas as pd
import re

infoRegex1 = r'^( ?[0-9]+\.)' 
inforRegex2 = re.compile('\(.*\)')


def qAbstractPreProcesse(a):
    tempText = toLower(a)
    tempText = removePunctuation(tempText)
    tempText = converteNumbers(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = stemWords(tempText)
    tempText = lemmatizeWords(tempText)
    # tempText = correctWords(tempText)
    return tempText

def qAuthorPreProcesse(a):
    tempText = toLower(a)
    lis = tempText.split(' ')
    names = ' '
    l = []
    for word in lis:
      if ',' in word:
          l.append(removePunctuation(word.replace(',','')))
    names = ' '.join(l)
    return names

def qInfoPreProcesse(i):
    # tempText = re.sub(infoRegex1, '', i)
    # tempText = ' '.join(inforRegex2.findall(tempText))
    # tempText = qAbstractPreProcesse(tempText)
    return i

In [23]:
import pandas as pd
def preprocesseQuery(dataFrame:pd.DataFrame):
    pdataFrame = pd.DataFrame(columns=['.I','data'])
    seriesDict:dict = {'.I':None, 'data':None} 
    seriesData = seriesDict.copy()
    for i in dataFrame.index:
        try:
            templist = []
            if not dataFrame.loc[i, '.W'] == '':
                templist.append(qAbstractPreProcesse(dataFrame.loc[i, '.W']))
            # if not dataFrame.loc[i, '.A'] == '':
            #     templist.append(qAuthorPreProcesse(dataFrame.loc[i, '.A']))
            # if not dataFrame.loc[i, '.N'] == '':
            #     templist.append(qInfoPreProcesse(dataFrame.loc[i, '.N']))

            seriesData['.I'] = i
            seriesData['data'] = ' '.join(templist)
            pdataFrame = pdataFrame.append(seriesData, ignore_index=True)
        except:
            print(i)
            raise 
    
    pdataFrame.set_index('.I', inplace=True)
    pdataFrame.fillna('', inplace=True)
    return pdataFrame


In [24]:
import pandas as pd
querydf = pd.read_csv('../../cacmData/cacmQueryCsv.csv')
querydf.set_index('.I', inplace=True)
querydf.fillna('', inplace=True)
querydf.head()

,.W,.A,.N
.I,,,
1,What articles exist which deal with TSS (Time ...,,"1. Richard Alexander, Comp Serv, Langmuir Lab ..."
2,I am interested in articles written either by ...,"Prieve, B. Pooch, U.","2. Richard Alexander, Comp Serv, Langmuir Lab ..."
3,Intermediate languages used in construction of...,,"3. Donna Bergmark, Comp Serv, Uris Hall (inter..."
4,I'm interested in mechanisms for communicating...,,4. Pavel Curtis (comm mech for disjoint proces...
5,I'd like papers on design and implementation o...,,5. Pavel Curtis (editing interfaces)


In [30]:
preprocessedQuery = preprocesseQuery(querydf)
preprocessedQuery.head()

,data
.I,
1,articl exist deal tss time share system oper s...
2,interest articl write either priev udo pooch
3,intermedi languag use construct multi target c...
4,interest mechan commun disjoint process possib...
5,like paper design implement edit interfac wind...


In [31]:
preprocessedQuery.to_csv('../../cacmData/cacmQueryPreprocessed.csv')

## index model

In [32]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import  cosine_similarity

data = pd.read_csv('../../cacmData/cacmDataPreprocessed.csv')
data.fillna('', inplace=True)
print(data.head())

# with open('../../../cacm/common_words', 'r') as f:
#     global commonWords
#     commonWords = f.read().split('\n')  

tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tfidfTable = tfidf.fit_transform(data['data'])


def search(query,n:int):
    querytfidf = tfidf.transform([query])
    cos = cosine_similarity(tfidfTable, querytfidf) 
    resultList = cos.argsort(axis=0)[-n:][::-1]
    ls = []
    ids = []
    for i in resultList:
        ids.append(data.loc[i,'.I'].to_list()[0])
        # print(data.loc[i,'.I'].to_list()[0])
    return ids


    .I                                               data          .B
0  1.0  preliminari report intern algebra languag perl...  1958-12-01
1  2.0    extract root repeat subtract digit comput sugai  1958-12-01
2  3.0     techniqu depart matrix program scheme friedman  1958-12-01
3  4.0           glossari comput engin program terminolog  1958-11-01
4  5.0                      two squar root approxim wadey  1958-11-01


In [33]:
queriesPath = '../../cacmData/cacmQueryPreprocessed.csv'
queriesData = pd.read_csv(queriesPath)


def queryingData(qDataFrame:pd.DataFrame, n):
    result = pd.DataFrame()
    resultDict:dict = {}
    resultDictCopy = resultDict.copy()
    for i in qDataFrame.index:
        try:
            tempList:list = search(qDataFrame.loc[i,'data'], n)
            for id in range(1,n+1):
                resultDictCopy[str(id)] = tempList[id - 1]
            result = result.append(resultDictCopy, ignore_index=True)
            resultDictCopy = resultDict.copy()
        except:
            print(i)
            raise
    return result

In [34]:
queriesResult = queryingData(queriesData, 10)
queriesResult.head()

,1,2,3,4,5,6,7,8,9,10
0,1938.0,1168.0,1071.0,1572.0,971.0,1908.0,2218.0,1410.0,2151.0,2371.0
1,1183.0,1781.0,1149.0,3078.0,597.0,1651.0,2526.0,282.0,2912.0,598.0
2,1134.0,1223.0,1496.0,2652.0,799.0,265.0,1162.0,3189.0,1870.0,1304.0
3,2939.0,2377.0,2376.0,3128.0,3101.0,3141.0,438.0,1439.0,3103.0,2450.0
4,2716.0,2321.0,2003.0,3011.0,2105.0,1829.0,2629.0,1272.0,2035.0,3144.0


## Evaluation

### precision

In [38]:
import pandas as pd
from sklearn.metrics import precision_score

def reSizeLists(l1:list, l2:list):
    if len(l1) < len(l2):
        l2 = l2[0:len(l1)]
    while len(l1) > len(l2):
        l1 = l1[0:len(l2)]

    return l1, l2, len(l1)


def precWithoutOrder(l1:list,l2:list):
    return len(set(l1).intersection(set(l2))) / len(l1)


def calcPrecisionAtK(resData:pd.DataFrame, qresData: pd.DataFrame):
    precisionsAtK:list = []
    precisionAtK:float
    for i in resData.index:
        precisionOnQuery = []
        resArray = resData.loc[i].to_numpy()
        qresArray = qresData.loc[qresData['.I'] == i+1, 'data'].to_numpy()
        resArray, qresArray, lenth = reSizeLists(resArray, qresArray)
        for lenI in range(lenth-1,lenth):
            tempRes:list = resArray[0:lenI+1].tolist()
            tempQRes:list = qresArray[0:lenI+1].tolist()
            precisionOnQuery.append(precision_score(tempQRes, tempRes, average='weighted', zero_division=0))
            #
            # if i == 0:
            #     print(tempRes)
            #     print(tempQRes)
            #     print(precisionOnQuery)
        # precisionOnQuery.append(precision_score(qresArray, resArray, average='weighted', zero_division=0))        
        try:
            precisionsAtK.append(sum(precisionOnQuery) / len(precisionOnQuery))
        except ZeroDivisionError: 
            precisionsAtK.append(0)
    precisionAtK = sum(precisionsAtK) / len(precisionsAtK)
    # print(precisionAtK)
    return precisionAtK

In [36]:
import pandas as pd
qrelsFrame = pd.read_csv('../../cacmData/cacmQRels.csv')

In [39]:
calcPrecisionAtK(queriesResult, qrelsFrame)

0.047070312499999996